______
<img style="float: right;" src="./images/headerlogo.png">
## Google Earth Engine Tutorial
### Nonlinear filters
_____

In previous noteboooks, we applied convolutional filters.  Those filter compute a linear combination of pixels in a neighbourhood(specified by the window shape) according to the weights specified by the kernel. Now, we will use nonlinear filter. [Median](https://en.wikipedia.org/wiki/Median_filter) and [mode](https://www.cs.washington.edu/research/metip/tutor/tutor.Filtering.html)  are examples of non linear filters.  <br>
Median filters compute the median (a non linear combination) in neighbourhood as well. An approach to apply nonlinear filters in GEE is by using the function *reduceNeighborhood*. 


In [1]:
import ee
from IPython.display import Image

ee.Initialize()

image = ee.Image('users/rosamaguilar/tutorial/subset')
uniformKernel = ee.Kernel.square(1)  # this means a 3x3 window

median = image.reduceNeighborhood(
  reducer = ee.Reducer.median(), 
  kernel = uniformKernel
)
 
# band names of the result image will have the suffix "_median"     
Image(url=median.getThumbUrl({'min': 0, 'max': 2048,'bands': 'b5_median,b3_median,b2_median'}))
 

In [2]:
uniformKernel = ee.Kernel.square(2)  # this means a 5x5 window

median = image.reduceNeighborhood(
  reducer = ee.Reducer.median(), 
  kernel = uniformKernel
)
 
# band names of the result image will have the suffix "_median"     
Image(url=median.getThumbUrl({'min': 0, 'max': 2048,'bands': 'b5_median,b3_median,b2_median'}))
 

In some cases, it is required to aggregate pixel values in a neighbourhood using the most common value (mode). This is special relevant for categorical maps that could be the result of a classification procedure.


In [3]:
# first, display the classified image in a map
image = ee.Image('users/rosamaguilar/tutorial/classified')
# define a palette as a string separate by ','
clas_col = ','.join(['000000','bdc3c7','1e8449','d68910','2ecc71','f9e79f'])   
Image(url=image.getThumbUrl({'min': 0, 'max': 5, 'palette':clas_col}))


In [4]:
# Let's calculate the mode of a classified image 

uniformKernel = ee.Kernel.square(1)  # this means a 3x3 window
mode = image.reduceNeighborhood(
  reducer = ee.Reducer.mode(), 
  kernel = ee.Kernel.square(2)
)

Image(url=mode.getThumbUrl({'min': 0, 'max': 5,'palette':clas_col}))

Filters can help enhancing visually the image and reducing noise. However, when the interest rely on objects (connected group of pixels)  present in the image rather than individual pixels, we may use [morphological operations](https://en.wikipedia.org/wiki/Mathematical_morphology).  These operations are useful to clean objects that can have gaps or are not well defined because of noise. 

Dilation and erosion are commonly applied to remote sensing images. The following lines apply dilation and erosion to a multi-spectral image. Later those operations are applied to a classified (categorical) image as well. In these examples, the *structuring element* is a square of size 3*3 pixels (square kernel of radio 1 pixel). *Opening* and *closing* can be executed by combining erosion and dilation. <br>


In [5]:
# dilation 
image = ee.Image('users/rosamaguilar/tutorial/subset')
# Dilate by taking the max in each 3x3 neighborhood.
imagemax = image.reduceNeighborhood(
  reducer = ee.Reducer.max(), 
  kernel = ee.Kernel.square(1)
)
Image(url=imagemax.getThumbUrl({'min': 0, 'max': 2048,'bands': 'b5_max,b3_max,b2_max'}))

In [6]:
# erosion
image = ee.Image('users/rosamaguilar/tutorial/subset')
# Dilate by takaing the max in each 3x3 neighborhood.
imagemin = image.reduceNeighborhood(
  reducer = ee.Reducer.min(), 
  kernel = ee.Kernel.square(1)
)
Image(url=imagemin.getThumbUrl({'min': 0, 'max': 2048,'bands': 'b5_min,b3_min,b2_min'}))

In [7]:
# dilation for a categorical image
#Image(url = imagemax.getThumbUrl({'min': 0, 'max': 5,'palette':clas_col}))
# dilation 
image = ee.Image('users/rosamaguilar/tutorial/classified')
# Dilate by takaing the max in each 3x3 neighborhood.
imagemax = image.reduceNeighborhood(
  reducer = ee.Reducer.max(), 
  kernel = ee.Kernel.square(1)
)
Image(url=imagemax.getThumbUrl({'min': 0, 'max': 5,'palette':clas_col}))

In [8]:
# erosion for a categorical image
#Image(url = imagemax.getThumbUrl({'min': 0, 'max': 5,'palette':clas_col}))
image = ee.Image('users/rosamaguilar/tutorial/classified')
# Dilate by takaing the max in each 3x3 neighborhood.
imagemax = image.reduceNeighborhood(
  reducer = ee.Reducer.min(), 
  kernel = ee.Kernel.square(1)
)
Image(url=imagemax.getThumbUrl({'min': 0, 'max': 5,'palette':clas_col}))

<i>Opening</i> can be computed applying a <i>dilation</i> over an <i>erosion</i><br>

In [9]:
# erosion for a categorical image
image = ee.Image('users/rosamaguilar/tutorial/classified')
imagemin = image.reduceNeighborhood(
  reducer = ee.Reducer.min(), 
  kernel = ee.Kernel.square(1))
# dilation 
imagemax = imagemax.reduceNeighborhood(
  reducer = ee.Reducer.max(), 
  kernel = ee.Kernel.square(1))

Image(url=imagemax.getThumbUrl({'min': 0, 'max': 5,'palette':clas_col}))

In [10]:
# In a similar way, to apply closing, apply a dilation followed by an erosion
image = ee.Image('users/rosamaguilar/tutorial/classified')
# dilation for a categorical image
imagemax = imagemax.reduceNeighborhood(
  reducer = ee.Reducer.max(), 
  kernel = ee.Kernel.square(1))
# erosion 
imagemin = imagemax.reduceNeighborhood(
  reducer = ee.Reducer.min(), 
  kernel = ee.Kernel.square(1))

Image(url=imagemin.getThumbUrl({'min': 0, 'max': 5,'palette':clas_col}))

You may want to try with a different structuring element, for example a disk by using a  kernel circle.